In [1]:
import torch 
from PIL import Image

from lavis.models import load_model_and_preprocess

/home/ommos92/miniconda3/envs/lavis_dev/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os

class WGVDataset(Dataset):
    def __init__(self, image_folder, csv_file, transform=None):
        
        self.image_folder = image_folder
        self.csv_file = csv_file
        self.transform = transform
        self.data = pd.read_csv(csv_file)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get the city name from the CSV file
        city = self.data.iloc[idx, 0]  # assuming the city name is in the first column

        # Find the image file in the image folder that contains the city name
        for filename in os.listdir(self.image_folder):
            if city in filename:
                img_path = os.path.join(self.image_folder, filename)
                break
        else:
            raise FileNotFoundError(f"No image file containing '{city}' found in the image folder")

        # Open the image file
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        state = self.data.iloc[idx, 1]
        country = self.data.iloc[idx, 2]
        continent = self.data.iloc[idx, 3]

        return image, state, country, continent

# Define the paths to the image folder and CSV file
image_folder = r"~/.cache/lavis/WGV/val/"
csv_file = "~/.cache/lavis/WGV/labels_list.csv"

# Define any additional transformations you want to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [16]:

# Create an instance of the LocationDataset
dataset = WGVDataset(image_folder, csv_file, transform)

# Create a DataLoader to load the data in batches
batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [17]:
# Load a sample image
image, state, country, continent = dataset[0]


FileNotFoundError: [Errno 2] No such file or directory: '~/.cache/lavis/WGV/val/'